<a href="https://colab.research.google.com/github/doukansurel/Retrieval-Augmented-Generation/blob/main/C%C3%BCmle_Penceresi_Alma_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Nest Asyncio

In [1]:
import nest_asyncio

nest_asyncio.apply()

### Bağımlılıkları Yükle

In [5]:
!pip install transformers


In [2]:
!pip install openai llama_index pypdf -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typ

### OpenAI API Key

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


## Gelişmiş Alma Yöntemi: Cümle Penceresi Alma
Yerleştirmelerimizde ince ayar yapmak, doğru bağlamı alma konusunda daha iyi olmamızı sağlamanın güçlü bir yoludur; ancak bir adım daha ileri giderek bağlama gerçekte bakma şeklimizi de geliştirebiliriz.

Bu gösterimde, SentenceWindowNodeParser ve meta veri değişimi fikrinden yararlanarak veri alımımızı bir sonraki seviyeye taşıyacağız.

Yüksek düzeyde yaptığımız şey basittir:

Belgemizi cümle bazında düğümlere ayrıştırıyoruz.
Sorgumuzla en alakalı cümle düğümlerini buluyoruz.
Bu temel cümle düğümünün etrafındaki bir "pencereye" dayalı olarak ek bağlam ekliyoruz.
Bu gelişmiş bağlamı LLM'miz için bağlam olarak kullanıyoruz!

In [6]:
from llama_index import ServiceContext, set_global_service_context
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding, HuggingFaceEmbedding
from llama_index.node_parser import SentenceWindowNodeParser, SimpleNodeParser
from llama_index import SimpleDirectoryReader

# pencere düğümü ayrıştırıcısı
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=6,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# basit düğüm ayrıştırıcı
simple_node_parser = SimpleNodeParser.from_defaults()

# temel Sorgu Motoru LLM
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

# base Embeddings model
embed_model_base = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en"
)

# fine-tuned ServiceContext
ctx = ServiceContext.from_defaults(
    llm=llm
)

# base ServiceContext
ctx_base = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model_base
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


belgelerimizi yükledikten sonra node_parser ve simple_node_parser'ımızı kullanarak düğümler oluşturalım.

In [8]:
documents = SimpleDirectoryReader(
        input_files=["/content/drive/MyDrive/RAG/Belgeler/risk-gruplarinin-belirlenmesine-ve-kredi-sinirlarina-iliskin-yonetmelik-1-1.pdf"]
).load_data()

In [9]:
nodes = node_parser.get_nodes_from_documents(documents)

In [10]:
base_nodes = simple_node_parser.get_nodes_from_documents(documents)

Düğümler için VectorStoreIndex'lerini oluşturabiliriz.

In [11]:
from llama_index import VectorStoreIndex

sentence_index = VectorStoreIndex(nodes, service_context=ctx)

In [17]:
sentence_index.storage_context.persist(persist_dir="sentence_index")

In [18]:
base_index.storage_context.persist(persist_dir="base_index")

In [12]:
base_index = VectorStoreIndex(base_nodes, service_context=ctx)

Bir sonraki adımda, cümlelerimizi (orijinal_metin) genişletilmiş bağlamlarımızla (pencere) değiştirecek olan MetadataReplacementPostProcessor'ımızı kuracağız.

 top_k (bu durumda 3) cümleyi alıyoruz ve ardından bunları çevreleyen bağlama dönüştürüyoruz.

In [19]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

In [20]:
window_response = query_engine.query("Kredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar neleri kapsar?")


In [21]:
window_response.response

'Kredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar, Kredi Riski Azaltım Tekniklerine İlişkin Tebliğde belirtilen kapsamlı finansal teminat yöntemi, fonlanmamış kredi korumaları ve bilanço içi netleştirme risk tutarı azaltımını içerir. Bu teknikler, belirli şartları sağlayan teminatların kullanılmasını ve risk tutarının azaltılmasını amaçlar. Ancak, kredi sınırları hesaplamasında kullanılan teminatlar, kredi riskine esas tutarın hesaplanmasında dikkate alınan teminatlarla aynı olmayabilir.'

In [22]:
window = window_response.source_nodes[0].node.metadata["window"]
sentence = window_response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: (2)  Merkezi  karşı  tarafla  yapılan  diğer  işlemlere  ilişkin  risk  tutarının  hesaplamasında,  bu  Yönetmelikte
belirtilen diğer taraflarla yapılan işlemlerin risk tutarlarının hesaplanmasına ilişkin hükümler uygulanır.
 (3) Bankanın üye kuruluş veya müşteri olduğu durumda takas işlemlerinden kaynaklanan risklerin karşı tarafı
Bankaların  Sermaye  Yeterliliğinin  Ölçülmesine  ve  Değerlendirilmesine  İlişkin  Yönetmeliğin  EK-4’ünde  yer
alan Merkezi Karşı Taraflardan Kaynaklanan Riskler İçin Sermaye Yükümlülüğünün Hesaplanmasına ilişkin
hükümler uyarınca belirlenir.
 (4)  Aralarında  aynı  risk  grubunda  değerlendirilmelerini  gerektiren  başka  bir  ilişki  bulunmaması  kaydıyla
sadece  takas  işlemleri  sebebiyle  birden  fazla  merkezi  karşı  tarafın  aynı  risk  grubunda  dikkate  alınmaları
zorunlu değildir.
 Kredi sınırları hesaplamalarında kredi riski azaltım tekniklerinin kullanılmasına ilişkin esaslar 
MADDE 12 
(1) Kredi sınırları hesaplamalarında, 6/9/2014 ta

Boru Hattımızı Değerlendirme
Hattımızı değerlendirmek için LlamaIndex'in değerlendirme araçlarından yararlanacağız.

GPT-4'ten yararlanarak QueryResponseDataset'imizi oluşturmak için DatasetGenerator kullanacağız

Oluşturulan veri kümesi öncekine benzer olacaktır; bu bir Soru/Bağlam veri kümesidir.



In [25]:
import random
from llama_index.evaluation import (
    DatasetGenerator,
    QueryResponseDataset,
)

# the number of nodes to evaluate
num_nodes_eval = 10

# selecting a random sample of nodes
sample_eval_nodes = random.sample(base_nodes, num_nodes_eval)

# setting up our GPT-4 powered evaluation context
eval_service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-4"))

# creating our dataset generator
dataset_generator = DatasetGenerator(
    sample_eval_nodes,
    service_context=eval_service_context,
    show_progress=True,
    num_questions_per_chunk=2,
)

<ipython-input-25-1880271f7cef>:17: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  dataset_generator = DatasetGenerator(


In [26]:
eval_dataset = await dataset_generator.agenerate_dataset_from_nodes()

100%|██████████| 2/2 [00:06<00:00,  3.10s/it]
/usr/local/lib/python3.10/dist-packages/llama_index/evaluation/dataset_generation.py:279: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [27]:
eval_dataset.save_json("llama_eval_qr_dataset.json")

In [28]:
eval_dataset = QueryResponseDataset.from_json("llama_eval_qr_dataset.json")

/usr/local/lib/python3.10/dist-packages/llama_index/evaluation/dataset_generation.py:93: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return cls(**data)


LlamaIndex tarafından sağlanan aşağıdaki standart değerlendirme metriklerini kullanacağız.

- CorrectnessEvaluator - [Code](https://github.com/run-llama/llama_index/blob/main/llama_index/evaluation/correctness.py)
- SemanticSimilarityEvaluator - [Code](https://github.com/run-llama/llama_index/blob/main/llama_index/evaluation/semantic_similarity.py)
- RelevancyEvaluator - [Code](https://github.com/run-llama/llama_index/blob/main/llama_index/evaluation/relevancy.py)
- FaithfulnessEvaluator - [Code](https://github.com/run-llama/llama_index/blob/main/llama_index/evaluation/faithfulness.py)

In [29]:
from llama_index.evaluation import (
    CorrectnessEvaluator,
    SemanticSimilarityEvaluator,
    RelevancyEvaluator,
    FaithfulnessEvaluator
)

evaluator_c = CorrectnessEvaluator(service_context=eval_service_context)
evaluator_s = SemanticSimilarityEvaluator(service_context=eval_service_context)
evaluator_r = RelevancyEvaluator(service_context=eval_service_context)
evaluator_f = FaithfulnessEvaluator(service_context=eval_service_context)

In [30]:
from llama_index.evaluation.eval_utils import get_responses, get_results_df
from llama_index.evaluation import BatchEvalRunner

max_samples = 15

eval_qs = eval_dataset.questions
ref_response_strs = [r for (_, r) in eval_dataset.qr_pairs]

In [31]:
query_engine = sentence_index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
pred_responses_finetuned_embeds = get_responses(
    eval_qs[:max_samples], query_engine, show_progress=True
)

100%|██████████| 15/15 [00:07<00:00,  2.13it/s]


In [32]:
base_index_base_embeddings = VectorStoreIndex(base_nodes, service_context=ctx_base)
base_embeddings_base_query_engine = base_index_base_embeddings.as_query_engine(
  similarity_top_k=3
)
base_pred_responses_base_embedings = get_responses(
    eval_qs[:max_samples], base_embeddings_base_query_engine, show_progress=True
)

100%|██████████| 15/15 [00:03<00:00,  4.63it/s]


In [33]:
import numpy as np

pred_response_strs_finetuned_embeds = [str(p) for p in pred_responses_finetuned_embeds]
base_pred_response_strs_base_embeds = [str(p) for p in base_pred_responses_base_embedings]

In [34]:
evaluator_dict = {
    "correctness": evaluator_c,
    "faithfulness": evaluator_f,
    "relevancy": evaluator_r,
    "semantic_similarity": evaluator_s,
}

batch_runner = BatchEvalRunner(evaluator_dict, workers=2, show_progress=True)

In [ ]:
base_eval_results_base_embeddings = await batch_runner.aevaluate_responses(
    queries=eval_qs[:max_samples],
    responses=base_pred_responses_base_embedings[:max_samples],
    reference=ref_response_strs[:max_samples],
)

In [ ]:
eval_results_finetuned_embeddings = await batch_runner.aevaluate_responses(
    queries=eval_qs[:max_samples],
    responses=pred_responses_finetuned_embeds[:max_samples],
    reference=ref_response_strs[:max_samples],
)

In [ ]:
results_df = get_results_df(
    [
        base_eval_results_base_embeddings,
        eval_results_finetuned_embeddings],
    ["Base Retriever w Base Embeddings", "Sentence Window Retriever w FT Embeddings"],
    ["correctness", "relevancy", "faithfulness", "semantic_similarity"],
)

In [ ]:
display(results_df.sort_values(by=['semantic_similarity'], ascending=False))